In [1]:
import chardet
from BeautifulSoup import BeautifulSoup 
import re
import os
from nltk.stem.porter import PorterStemmer
from langdetect import detect
import pymorphy2
import regex
import pickle
import codecs
import docx
import numpy as np
import pandas as pd
import html2text
import ipyparallel as ipp
# ipcluster nbextension enable

In [13]:
query_with_digit = [8, 27, 32, 51, 64, 68, 69, 71, 83, 91, 101, 110, 120, 122, 131, 133, 140, 143, 148, \
                   174, 184, 189, 190, 196, 203, 210, 212, 221, 230, 233, 235, 236, 256, 268, 272, 283, \
                    294, 299, 306, 312]

bad_query = [36, 211, 61, 87, 264, 324, 328, 329]
print len(query_with_digit)

sample_submit = pd.read_csv("sample.csv").values
#номер запроса - список документов для ранжирования
docs_to_reparse = []
for q in sample_submit:
#     print q
    if q[0] in bad_query:
        docs_to_reparse.append(q[1])
    else:
        pass
    
print docs_to_reparse
print len(docs_to_reparse)

40
[69, 212, 416, 492, 496, 728, 777, 2141, 2183, 2282, 2449, 2599, 3001, 3002, 3025, 3051, 3646, 4258, 4282, 4373, 4856, 4934, 5090, 5091, 5134, 6103, 7371, 7602, 8116, 8117, 8292, 8309, 8364, 8525, 8678, 8708, 9992, 10200, 10299, 10319, 10331, 12759, 13400, 13665, 14267, 14299, 14317, 15172, 15280, 15450, 15548, 15552, 16059, 16768, 16849, 17727, 17986, 17999, 18484, 18672, 18898, 19401, 19715, 21679, 21743, 20512, 20513, 20543, 20710, 20833, 20838, 21909, 21949, 22106, 22122, 22725, 23058, 23235, 23281, 23460, 23806, 24925, 24926, 25541, 26384, 26751, 87, 841, 1111, 1780, 2545, 1989, 1898, 2406, 2605, 3068, 3478, 3632, 3975, 4489, 4557, 5019, 5284, 5329, 5355, 5523, 5743, 6052, 6562, 6794, 7124, 7453, 7511, 7527, 8209, 8875, 8876, 9010, 9089, 9142, 9250, 9660, 9691, 9712, 9768, 9813, 9814, 9984, 10063, 10359, 10365, 11064, 11618, 11757, 11788, 11825, 12230, 12348, 12359, 12362, 12369, 12464, 12502, 12806, 12998, 13931, 14154, 14370, 14488, 14531, 14545, 14549, 14799, 15368, 16176, 1

In [3]:
def create_idx_query_dict(idx_query_file):
    #по индексу выдает запрос, причем обработанный
    idx_query = {}
    f = open(idx_query_file, "r")
    for line in f:
        l = line.decode("utf-8").strip().split()
        idx_query[int(l[0])] = " ".join(l[1:])
    return idx_query

IDX_QUERY = create_idx_query_dict("./queries.numerate_test.txt")

for q in query_with_digit:
    print IDX_QUERY[q]

в чем различие между ос андроид 5 1 и андроид 6 0
где проходит выставка индийских товаров с 2 по 11 сентября в красноярске 2016
дата выборов в госдуму в 2016 году
как быстро скинуть 10 кг
как доехать наб реки мойка 50
как заполнить в налоговую 1 рн образец
как заполняем книгу рро 2016 г
как заряжать айфон 4 без провода
как настроить мобильные данные на tele 2
как отдыхаем на курбан байрам 2016 в казани
как печатать на компьютере в формате а 5
как пойти в спайдермен 3 миссию ящер 1
как посчитать площадь стен перегородок в м3
как похудеть за неделю на 10 кг в домашних условиях
как правильно установить зажигание на тойоте харриер двигатель 5 с
как приготовить вкусную запеканку из рожков
как приручить дракона 3
как продифференцировать функцию алгебра 11 класс
как решать неравенство 8 кл подробное объяснение
как стрелять без отдачи и разброса в кс 1 6
как узаконить самострой в украине в 2016 году
как устанавливать моды на fallout 3
как установить мелодию на контакт самсунг галакси j1 мини
к

In [14]:
h = html2text.HTML2Text()
h.ignore_links = True
h.ignore_images = True
h.ignore_emphasis = True
def bprint(l):
    for x in l:
        print x
    print

#определяет, является ли файл pdf, если да - извлекает из него текст в файл txt_file_name
def isPDF(file_name, txt_file_name):
#     print file_name
    t= !pdftotext -enc "UTF-8" {file_name} {txt_file_name}
    not_pdf = 0
    for er in t:
        if er[:14] != 'Syntax Warning':
            not_pdf += 1
    print "is not pdf??", len(t), not_pdf
    if not_pdf:
        return False
    return True

def EncodeDetect(file_name):
    res = chardet.detect(file_name)
    print res
    if res['confidence'] > 0.1:
        return res['encoding']
    return False


def visible(element):
    if element.parent.name in [u'style', u'script', u'[document]', u'head', u'title']:
        return False
#     if re.match('<!--.*-->', str(element)):
#         return False
#     elif re.match('\n', str(element)):
#         return False
    elif re.match('[&]+.*', str(element), re.U):
        return False
    return True

def isDoc(file_name):
# не буду париться с кодировкой для doc - как-то нипонятна, доков всего ~100
    print file_name
    t= !pdftotext -enc "UTF-8" {file_name} {txt_file_name}
    if len(t) > 1:
        return False
    return True
    try:
        doc = docx.Document(file_name)
        fullText = []
        for para in doc.paragraphs:
            fullText.append(para.text)
        return ' '.join(fullText)
    except Exception as ex:
        print ex
        return False

# проверка на html файл, в случае успеха возвращается текст
def isHTML_simple(file_name):
    try:
        html = open(file_name, "r")
    #     print EncodeDetect(html.read())
        encode = EncodeDetect(html.read())
        if encode:
            print "try html2text"
            text = h.handle(open(file_name).read().decode(encode))
            if text:
                print "OK"
                return text
            return " ".join(visible_texts)

    except Exception as ex:
        print ex    
        return False
        
        
# проверка на html файл, в случае успеха возвращается текст
def isHTML(file_name):
    html = open(file_name, "r")
#     print EncodeDetect(html.read())
    encode = EncodeDetect(html.read())
    if encode:
        html = codecs.open(file_name, "r", encode)
    else:
        print "cant encoding", file_name
        html = open(file_name, "r")
    try:
        soup = BeautifulSoup(html.read())
        texts = soup.findAll(text=True)
        visible_texts = filter(visible, texts)
        print len(visible_texts)
        if len(visible_texts) == 0:
            print "try html2text"
            text = h.handle(open(file_name).read().decode(encode))
            if text:
                print "OK"
                return text
        return " ".join(visible_texts)
    
    except Exception as ex:
        print "html_check_failure", ex
        try:
            html = open(file_name, "r")
            soup = BeautifulSoup(html.read())
            texts = soup.findAll(text=True)
            visible_texts = filter(visible, texts)
            print len(visible_texts)
            if len(visible_texts) == 0:
                print "try html2text"
                text = h.handle(open(file_name).read().decode(encode))
                if text:
                    print "OK"
                    return text
            return " ".join(visible_texts)
        
        except Exception as ex:#dsg
            print "Double error in html check"
            # h.ignore_tables = True
            text = h.handle(open(file_name).read().decode(encode))
            if text:
                print "OK"
                return text
            return False
    
    
# проверка на html файл, в случае успеха возвращается текст
# def isHTML_debag(file_name):
#     html = open(file_name, "r")
# #     print EncodeDetect(html.read())
#     encode = EncodeDetect(html.read())
#     if encode:
#         html = codecs.open(file_name, "r", encode)
#     else:
#         print "cant detect encoding", file_name
#         html = open(file_name, "r")
#     try:
#         soup = BeautifulSoup(html.read())
#         texts = soup.findAll(text=True)
#         visible_texts = filter(visible, texts)
#         print len(visible_texts)
#         for v in visible_texts:
#             print v
# #         return " ".join(visible_texts)
#     except Exception as ex:
#         print "html_check_failure", ex
#         return False

def create_URL_IDX_dict(url_index_file):
    url_idx = {}
    f = open(url_index_file, "r")
    for line in f:
        l = line.strip().split()
        url_idx[l[1]] = l[0]
    return url_idx

def get_idx_of_doc(doc_name, url_idx):
    doc = open(doc_name, "r")
    i = 0
    for line in doc:
        #новое имя документа - индекс его урла
        strip_line = line.strip()
#         print strip_line
        if strip_line[0] == '\'':
            new_file_name = url_idx[strip_line[1:-1]]
        else:  
            new_file_name = url_idx[strip_line]
        print new_file_name
        
        return new_file_name, strip_line

lang = {}
Porter = PorterStemmer()
RusLem = pymorphy2.MorphAnalyzer()
UkLem = pymorphy2.MorphAnalyzer(lang='uk')

#разбиваем текст (unicode) на токены, приводим к нижнему регистру и применяем стэмминг
def TextParse(text):
    #цифры стоит оставлять
    tokens = re.findall('[\w]+',text.lower(), re.U)
    #теперь стемминг зависит от языка. Надо его определить.
    if detect(text) == u'ru':
        print "rus"
        if 'ru' in lang:
            lang['ru'] += 1
        else:
            lang['ru'] = 1
        
        tokens = [RusLem.parse(token)[0].normal_form for token in tokens]
    elif detect(text) == u'uk':
        if 'uk' in lang:
            lang["uk"] += 1
        else:
            lang["uk"] = 1
        
        tokens = [UkLem.parse(token)[0].normal_form for token in tokens]
    elif detect(text) == u'en':
        if 'en' in lang:
            lang["en"] += 1
        else:
            lang["en"] = 1
        tokens = [Porter.stem(token) for token in tokens]
    return tokens
        

def DocParse(file_name):
    #для начала определим формат файла
    if isPDF(file_name, "tmp.txt"):
        print "IT IS A PDF"
        #если это пдф-ка, то она теперь преобразована в файл temp.txt в кодировке UTF-8
        #теперь выгружаем его и продолжаем парсить
        tmp_file = open("tmp.txt", "r")
        text = " ".join([line.strip() for line in tmp_file]).decode('utf-8')
        tokens = TextParse(text)
    else:
        text = isHTML(file_name)
#         print text[:len(text)]
#         print "df", text, "sf"
        if text:
            print "IT IS A HTML"
            tokens = TextParse(text)
            bprint(tokens[10:20])
        else:
#             text = isDoc(file_name)
#             if text:
#                 print "IT IS A DOC"
#                 tokens = TextParse(text)
#             else:
            print "SRTANGE_FORMAT:", file_name
            
    return tokens

def DocParse_withoutPDF(file_name):
    #для начала определим формат файла
    text = isHTML(file_name)
#         print text[:len(text)]
#         print "df", text, "sf"
    if text:
        print "IT IS A HTML"
        tokens = TextParse(text)
        bprint(tokens[10:15])
    else:
#             text = isDoc(file_name)
#             if text:
#                 print "IT IS A DOC"
#                 tokens = TextParse(text)
#             else:
        print "STRANGE_FORMAT:", file_name
            
    return tokens

def load_list(file):
    res = []
    for l in file:
        res.append(l[:-1])
    return res

# url_idx - словарь, сопоставляющий каждому урлу свой индекс
def Preprocessing(url_idx):
    already_processed = []
    try:
        already_done_file = open("already_done_new", "r")
        already_processed = load_list(already_done_file)
    except:
        pass
#     print already_processed
    already_done_file = open("already_done_new", "a")
    
    cnt_url_without_idx = 0
    cnt_done_files = len(already_processed)
    for dir_name in os.listdir("./data"):
        for file_name in os.listdir("./data/" + dir_name):
            if "./data/"+ dir_name + "/" + file_name in already_processed:
#                 print "./data/"+ dir_name + "/" + file_name +" already processed"
                continue
            try:
                new_file_name, url = get_idx_of_doc("./data/"+ dir_name + "/" + file_name, url_idx)
                if int(new_file_name) not in docs_to_reparse:
                    continue
                #для этого урла есть индекс - можем парсить.
                if url[-4:] == '.doc' or url[-4:] == ".pdf":
                    print "doc or pdf"
                    continue
                tokens = DocParse_withoutPDF("./data/"+ dir_name + "/" + file_name)
#                 print tokens

                if len(tokens) > 0:
                    print "./data/"+ dir_name + "/" + file_name
                    print "tokens in file:", len(tokens) 
                    if not os.path.exists("./data_tokens_new/"+ dir_name ):
                        os.makedirs("./data_tokens_new/"+ dir_name )
                    
                    with open("./data_tokens_new/"+ dir_name + "/" + new_file_name, 'wb') as res_file:
                        pickle.dump(tokens, res_file)
                        
                    already_done_file.write("./data/"+ dir_name + "/" + file_name + "\n")
                    already_done_file.flush()
#                     doc_place.write(new_file_name + " " + dir_name + "\n")
#                     doc_place.flush()
                    
                    print "New_File: ", dir_name + "/"+ new_file_name 
                    cnt_done_files += 1
                    print "Processed files count:", cnt_done_files
                    
                else:
                    "zero tokens in file", "./data/"+ dir_name + "/" + file_name
                    
            except Exception as inst:
                print inst
                print "ERROR:", "./data/"+ dir_name + "/" + file_name
                cnt_url_without_idx += 1
    
    print "CNT_URL_WITHOUT_IDX:", cnt_url_without_idx

In [15]:
#номер документа: номер папки
doc_place = open("doc_place", "a")
URL_IDX = create_URL_IDX_dict("./urls.numerate.txt")

In [16]:
Preprocessing(URL_IDX)

22739
23474
10114
10347
24193
20753
19520
1076
25716
19359
1426
26299
17921
4065
18788
26440
17038
{'confidence': 0.8968333719856458, 'encoding': 'windows-1251'}
115
IT IS A HTML
не
задана
тип
баннера
background

./data/65/-706020461305818783
tokens in file: 4473
New_File:  65/17038
Processed files count: 3059
23915
26363
10839
7018
25968
5853
1190
26360
9629
607
20000
23498
24083
21575
5656
22575
17332
4500
20647
24984
19087
22627
7262
14217
25599
15478
20851
24167
3972
14630
15503
20270
25480
20790
5988
18158
21842
5395
6439
14288
4036
15645
12862
508
6971
17908
772
21524
5060
7670
8149
6326
22164
21913
{'confidence': 0.6055976194035708, 'encoding': 'windows-1251'}
124
IT IS A HTML
rus
править
дать
домой
найти
дать

./data/65/-2186160311847132101
tokens in file: 24
New_File:  65/21913
Processed files count: 3060
9429
12524
18617
16028
9225
5096
23892
23727
26390
24805
11335
17132
14547
19372
24250
16829
13177
990
25105
21673
1605
13249
6503
23199
19764
16928
25157
25287
8178
26592
10

In [78]:
def check_file(file_name):
    with open("./data_tokens/" + file_name, 'rb') as res_file:
        tokens = pickle.load(res_file)
    print len(tokens)
    for t in tokens:
        print t

In [121]:
check_file("23/19458")

13709
đ
ldquo
đťđ
đ
đ
đ
ń
đ
đşđžđ
đžđźđ
đşđ
đ
lsquo
đ
đˇđ
đľń
đ
yuml
đžđťđ
ń
sbquo
đ
đşđ
đ
rdquo
ń
fnof
ń
hellip
đžđ
đ
đžń
ń
sbquo
ń
oelig
đ
đľń
hellip
đ
đž
đ
scaron
ń
fnof
đťń
oelig
ń
sbquo
ń
fnof
ń
euro
đ
đąđżđžń
euro
ń
sbquo
đ
mdash
đ
đžń
euro
đžđ
ń
oelig
đľ
đ
đşđžđťđžđłđ
ń
đ
rsquo
đźđ
ń
euro
đľ
select
a
page
đ
ldquo
đťđ
đ
đ
đ
ń
đ
đşđžđ
đžđźđ
đşđ
đ
lsquo
đ
đˇđ
đľń
đ
yuml
đžđťđ
ń
sbquo
đ
đşđ
đ
rdquo
ń
fnof
ń
hellip
đžđ
đ
đžń
ń
sbquo
ń
oelig
đ
đľń
hellip
đ
đž
đ
scaron
ń
fnof
đťń
oelig
ń
sbquo
ń
fnof
ń
euro
đ
đąđżđžń
euro
ń
sbquo
đ
mdash
đ
đžń
euro
đžđ
ń
oelig
đľ
đ
đşđžđťđžđłđ
ń
đ
rsquo
đźđ
ń
euro
đľ
đ
rsquo
đ
oelig
đžń
đşđ
đľ
đżđžđťđ
ń
dagger
đľđšń
đşđ
đľ
đˇđ
đąń
euro
đ
đťđ
ń
fnof
đż
hellip
đ
rsquo
đ
oelig
đžń
đşđ
đľ
ń
đžń
sbquo
ń
euro
ń
fnof
đ
đ
đ
đşđ
đşđ
ń
euro
đ
ń
sbquo
hellip
đ
yuml
đžđąđľđ
đžđ
đžń
đ
đ
ń
đ
đžđšđ
đ
ń
đľđ
đžđš
đ
đ
đ
ń
đľ
hellip
đ
scaron
đ
đş
ń
đžđžđąń
permil
đ
đľń
sbquo
ń
fnof
đżń
euro
đ
đ
đťđľđ
đ
đľ
đ
hellip
đ
yuml
đžđ
ń
dagger
ń
fnof
ń
sbquo
đşđ
đź
ń
euro
ń
fnof
đ

In [284]:
tokens = DocParse("./data/10/1021518709650121966")
print len(tokens)
for t in tokens:
    print t

./data/10/1021518709650121966
{'confidence': 0.0, 'encoding': None}
cant encoding ./data/10/1021518709650121966
Package not found at './data/10/1021518709650121966'
SRTANGE_FORMAT: ./data/10/1021518709650121966


UnboundLocalError: local variable 'tokens' referenced before assignment

In [259]:
isDoc("./data/1/5933403300958279234")

Package not found at './data/1/5933403300958279234'


False

In [279]:
import textract
text = textract.process("./data/1/5933403300958279234")

TypeError: decode() argument 1 must be string, not None

In [262]:
f = open("./data/1/5933403300958279234", "r")

In [277]:
sample_submit = pd.read_csv("sample.csv").values
docs_to_ranking = set(sample_submit[:, 1])
print len(docs_to_ranking)

26803


In [278]:
print len(URL_IDX)

26803


In [276]:
print sample_submit


[[    1   222]
 [    1   244]
 [    1   842]
 ..., 
 [  339 26603]
 [  339 26609]
 [  339 26631]]


In [286]:
!antiword "./data/10/1021518709650121966"

./data/10/1021518709650121966 is not a Word Document.


http://fikio.ru/?tag=teoriya-i-istoriya-kulturyi 6503
{'confidence': 0.99, 'encoding': 'utf-8'}
IT IS A HTML
rus
tokens in file: 66880
New_File:  65/6503
Processed files count: 106

http://avtosovet.com.ua/avtoporada/shho-take-galma-abs 1723
{'confidence': 0.43199280319585814, 'encoding': 'ISO-8859-2'}
cant encoding ./data/65/-1818941932081991926
SRTANGE_FORMAT: ./data/65/-1818941932081991926
local variable 'tokens' referenced before assignment
ERROR: ./data/65/-1818941932081991926

http://versii.cv.ua/yurydychni/pratsivnyk-u-den-zvilnennya-ne-otrymav-trudovu-knyzhku-scho-robyty/4444.html 24261
{'confidence': 1.0, 'encoding': 'ascii'}
SRTANGE_FORMAT: ./data/2/1909137208908550223
local variable 'tokens' referenced before assignment
ERROR: ./data/2/1909137208908550223

http://kulichki.com/moshkow/DETEKTIWY/CHEJZ/onsvoep.txt 11049
{'confidence': 0.9846769619262623, 'encoding': 'KOI8-R'}
IT IS A HTML
tokens in file: 64
New_File:  2/11049

42/9895

In [154]:
rus_dict = []
with open("./unigrams.cyr.lc", "r") as dict_file:
    for line in dict_file:
        rus_dict.append(line.strip().split()[0].decode("utf-8"))

In [155]:
for w in rus_dict[:10]:
    print w

print len(rus_dict)

в
и
на
не
с
что
по
а
как
к
164465


In [156]:
def bprint(l):
    for x in l:
        print x
    print
    

#разбиваем текст (unicode) на токены, приводим к нижнему регистру и применяем нормализацию
def TextParse_(text):
    tokens = re.findall('[^\W\d]+', text.lower(), re.U)
    #теперь стемминг зависит от языка. Надо его определить.
    if detect(text) == u'ru':        
        tokens = [RusLem.parse(token)[0].normal_form for token in tokens]
    elif detect(text) == u'uk':
        tokens = [UkLem.parse(token)[0].normal_form for token in tokens]
    elif detect(text) == u'en':
        tokens = [Porter.stem(token) for token in tokens]
    return tokens


def create_idx_query_dict(idx_query_file):
    #по индексу выдает запрос, причем обработанный
    idx_query = {}
    f = open(idx_query_file, "r")
    for line in f:
        l = line.decode("utf-8").strip().split()
        idx_query[int(l[0])] = TextParse_(" ".join(l[1:]))
    return idx_query
IDX_QUERY = create_idx_query_dict("./queries.numerate.txt")


In [157]:
for query_tokens in IDX_QUERY.values():
    bprint(query_tokens)
    for token in query_tokens:
        print token in rus_dict

rfr
cyznm
jnntrb
gjl
ukfpfvb

False
False
False
False
False
банк
выставить
счёт
за
плата
за
использование
кредит
сверх
лимит
законно
ли
это

True
True
False
True
True
True
True
True
True
True
True
True
True
в
какой
срок
в
это
год
сбор
зимний
яблоко
в
волгоград

True
True
True
True
True
True
True
True
True
True
True
в
какой
проект
занятый
дмитрий
шепелевый

True
True
True
True
True
False
в
какой
возраст
стерилизовать
кошка

True
True
True
True
True
в
какой
полушарие
находиться
россия

True
True
True
True
True
в
чем
заключаться
работа
администратор
офис

True
True
True
True
True
True
в
чем
различие
между
ос
андроид
и
андроид

True
True
True
True
True
True
True
True
видеть
ли
вы

True
True
True
в
сколько
месяц
можно
брать
котёнок
от
кошка

True
True
True
True
True
False
True
True
где
вбить
дать
доверенный
лицо
получать
денежный
средство
через
пко
в
с

True
True
True
True
True
True
True
True
True
False
True
True
где
гай
стоять
ижевск

True
True
True
False
где
давать
земля
многодетный
в
усм

In [2]:
#!/usr/bin/env python
import sys
from contextlib import closing

import lxml.html as html # pip install 'lxml>=2.3.1'
from lxml.html.clean        import Cleaner
from selenium.webdriver     import Firefox         # pip install selenium
from werkzeug.contrib.cache import FileSystemCache # pip install werkzeug

cache = FileSystemCache('.cachedir', threshold=100000)

url = sys.argv[1] if len(sys.argv) > 1 else "http://stackoverflow.com/q/7947579"


# get page
page_source = cache.get(url)
if page_source is None:
    # use firefox to get page with javascript generated content
    with closing(Firefox()) as browser:
        browser.get(url)
        page_source = browser.page_source
    cache.set(url, page_source, timeout=60*60*24*7) # week in seconds


# extract text
root = html.document_fromstring(page_source)
# remove flash, images, <script>,<style>, etc
Cleaner(kill_tags=['noscript'], style=True)(root) # lxml >= 2.3.1
print root.text_content() # extract text

WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 


In [20]:
import html2text
h = html2text.HTML2Text()
h.ignore_links = True
h.ignore_images = True
h.ignore_emphasis = True
# h.ignore_tables = True
print(h.handle(open("./data/65/444704432132796763").read().decode('windows-1251')))

http://i.ovkuse.ru/blogs/recepty-iz-interneta/20-samyh-vkusnyh-nachinok-dlja-
ruleta-iz-lavasha.html

  * Войти
  *   *   * 

Регистрация

  * Уведомления

  * Каталог рецептов

Каталог рецептов всего 117 075 рецептов

    * Блюда из мяса
    * Блюда из птицы
    * Блюда из рыбы
    * Видео рецепты
    * Детские рецепты
    * Диетические
    * Для мультиварок
    * Кухни стран мира
    * На скорую руку
    * Новогодние рецепты
    * Пасха
    * Постные рецепты
    * Завтрак
    * Обед
    * Ужин
    * Все рецепты
    * Все ингредиенты
    * Меню на праздничный стол
    * Выбор потребителей
    * ★ Популярные рецепты
    * Рейтинг рецептов
    * Статьи партнеров

    * Выпечка
      * блины, 
      * запеканки, 
      * кексы, 
      * лаваш, 
      * лепешки, 
      * манник, 
      * оладьи, 
      * печенье, 
      * пирожки, 
      * пирожные, 
      * пироги, 
      * пицца, 
      * сырники, 
      * тесто, 
      * торты, 
      * хлеб, 
      * шарлотка, 
      * шарлотка с ябло

In [14]:
?html2text.HTML2Text()